In [1]:
import numpy as np
import time
import os
import h5py
import inspect
from tqdm import tqdm
import sys
import math

import presto
from presto import lockin, utils
from presto.hardware import AdcFSample, AdcMode, DacFSample, DacMode


In [2]:
############################################################################
# Saving methods

# Save script function
def save_attr(folder, file, sample_, myrun_, myrun_attrs_):
    if not os.path.isdir(folder):
        os.makedirs(folder)
    with h5py.File(os.path.join(folder, file), "a") as savefile:
        run_str = "{}".format(myrun_)
        # Assicurati che il gruppo esista
        if run_str not in savefile:
            savefile.create_group(run_str)
        for key, val in myrun_attrs_.items():
            savefile[run_str].attrs[key] = val
    print("Saved run attributes.")


# Save data function
def save_data(folder, file, sample, myrun, freq, freq_pump, pump_pwr, pump_phase, df, usb_arr):
    if not os.path.isdir(folder):
        os.makedirs(folder)

    # Open the save file (.hdf5) in append mode
    with h5py.File(os.path.join(folder, file), "a") as savefile:
        # String as handles
        freq_data_str = "{}/freq comb".format(myrun)
        freq_pump_data_str = "{}/freq pumps".format(myrun)
        pump_pwr_data_str = "{}/pump pwr sweep".format(myrun)
        pump_phase_data_str = "{}/pump phase sweep".format(myrun)
        df_data_str = "{}/df".format(myrun)
        usb_data_str = "{}/USB".format(myrun)

        # Write data to datasets
        savefile.create_dataset(freq_data_str, (np.shape(freq)),
                                dtype=float, data=freq)
        savefile.create_dataset(freq_pump_data_str, (np.shape(freq_pump)),
                                dtype=float, data=freq_pump)
        savefile.create_dataset(pump_pwr_data_str, (np.shape(pump_pwr)),
                                dtype=float, data=pump_pwr)
        savefile.create_dataset(pump_phase_data_str, (np.shape(pump_phase)),
                                dtype=float, data=pump_phase)
        savefile.create_dataset(df_data_str, (np.shape(df)),
                                dtype=float, data=df)
        savefile.create_dataset(usb_data_str, (np.shape(usb_arr)),
                                dtype=complex, data=usb_arr)

        # Write dataset attributes
        savefile[freq_data_str].attrs["Unit"] = "Hz"
        savefile[freq_pump_data_str].attrs["Unit"] = "Hz"
        savefile[pump_pwr_data_str].attrs["Unit"] = "fsu"
        savefile[pump_phase_data_str].attrs["Unit"] = "rad"
        savefile[df_data_str].attrs["Unit"] = "Hz"
        savefile[usb_data_str].attrs["Unit"] = "fsu complex"



In [3]:
Ym_str = time.strftime("%Y-%m")
meas_type   = 'Scattering'
save_folder = r'I:/LKiPA-Data/{}/{}'.format(Ym_str, meas_type)
myrun       = time.strftime("%Y-%m-%d_%H_%M_%S")
save_file   = r"{}.hdf5".format(myrun)  

sample      = 'LKIPA'
meas_type   = 'Scattering'
atten       = 80
temperature = 0.0096

# Type of measurement
reverse = False
fwd_and_rev = False

ADDRESS = '130.237.35.90'
PORT    = 42873
Box     = 'Presto DELTA'

input_port   = 5
output_port  = 8
pump_port    = 2
bias_port    = 2

# Pseudorandom noise (only when working with small amplitudes)
dither = False

# DAC current
DAC_CURRENT  = 32_000  # µA

# DC bias value
bias_val     = 4.50   # FS  From calibration   4.50 V --> f0= 4424855567.651496 Hz  

# MEASUREMENT PARAMETERS
# NCO frequencies
fNCO = 4.31e9
fNCO_pump = 8.6e9
# Bandwidth in Hz
_df_arr = np.array([25e3, 10e3, 7.5e3]) # np.array([1e6, 750e3, 500e3, 250e3, 100e3, 75e3, 50e3, 25e3, 10e3, 7.5e3, 5e3, 2.5e3, 1e3]) #np.array([250e3, 100e3, 50e3, 10e3, 1e3]) 

# Number of pixels
_Npix = 5000
Npix_arr = _Npix*np.ones_like(_df_arr)
Navg = 1
# Number of pixels we discard
Nskip = 0
# Number of frequencies of the frequency comb
nr_sig_freqs = 95

# SIGNAL PARAMETERS
# Signal output amplitude from Presto [FS] (Never exceed 0.05FS for OUTPORT [input of JPA]!!! - actually 0.1 still is fine)
amp_sig = 0.01

# RF PUMP PARAMETERS
# Pump frequency in Hz
_f0 = 4_424_855_567 #.651496
_fp_center = 2*_f0
# Number of pumps
nr_pump_freqs = 1
# Pump amplitude sweep (FS units between 0 and 1)
pump_amp_min = 0.0
pump_amp_max = 0.1
# nr_pump_amps = 11
# pump_amp_arr = np.linspace(pump_amp_min, pump_amp_max, nr_pump_amps)
pump_amp_arr = np.array([0.0, 0.1, 0.2, 0.3]) 
nr_pump_amps = len(pump_amp_arr)
k_pmp=0
k_array      = np.array([k_pmp])
# Pump 1 phase sweep
nr_pump_phases = 1
pump_phase_arr = np.linspace(0, 2 * np.pi, nr_pump_phases)

# Propagation Phase Drift
input_phases = np.zeros(nr_sig_freqs)
dPhiDrift_df = 1.8869636024e-6  # rad/Hz - Measured Drift in phase per Hz
# dPhiDrift_df = 1.8964498850e-6  # rad/Hz - Measured Drift in phase per Hz for df=1MHz

# Instantiate lockin device
with lockin.Lockin(address=ADDRESS,
                   port=PORT,
                   adc_mode=AdcMode.Mixed,
                   adc_fsample=AdcFSample.G2,
                   dac_mode=[DacMode.Mixed04, DacMode.Mixed02, DacMode.Mixed02, DacMode.Mixed02],
                   dac_fsample=[DacFSample.G10, DacFSample.G6, DacFSample.G6, DacFSample.G6],
                   ) as lck:
    # Start timer
    t_start = time.strftime("%Y-%m-%d_%H_%M_%S")

    # Print Presto version
    print("Presto version: " + presto.__version__)

    # DAC current
    lck.hardware.set_dac_current(output_port, DAC_CURRENT)

    # Data
    usb_arr = np.zeros((len(_df_arr), nr_pump_amps, nr_pump_phases, nr_sig_freqs, nr_sig_freqs), dtype=complex)

    # Bandwidth array and frequency combs arrays (to save)
    df_arr = np.zeros_like(_df_arr)
    fs_comb_arr = np.zeros((len(df_arr), nr_sig_freqs))
    fp_comb_arr = np.zeros((len(df_arr), nr_pump_freqs))
    fp_center_arr = np.zeros_like(_df_arr)

    # Set DC bias value
    lck.hardware.set_dc_bias(bias_val, bias_port)
    lck.hardware.sleep(1.0, False)

    # Configure mixer for the JPA pump
    lck.hardware.configure_mixer(freq=fNCO_pump,
                                 out_ports=pump_port,
                                 sync=False,
                                 )

    # Configure mixer just to be able to create output and input groups
    lck.hardware.configure_mixer(freq=fNCO,
                                 in_ports=input_port,
                                 out_ports=output_port,
                                 sync=True,
                                 )

    # Create output group for the JPA pump frequency
    og_pump = lck.add_output_group(ports=pump_port, nr_freq=nr_pump_freqs)

    # Create output group for the signal
    og = lck.add_output_group(ports=output_port, nr_freq=1)
    og.set_amplitudes(amp_sig)
    og.set_phases(phases=0.0,
                  phases_q=-np.pi / 2)

    # Create input group
    ig = lck.add_input_group(port=input_port, nr_freq=nr_sig_freqs)

    # Add pseudorandom noise if needed
    lck.set_dither(dither, output_port)

    lck.apply_settings()
    lck.hardware.sleep(1e-4, False)

    # Display nice progress bar
    with tqdm(total=(len(_df_arr) * nr_pump_amps * nr_pump_phases * nr_sig_freqs), ncols=80) as pbar:

        # Bandwith loop
        for df_idx, df_val in enumerate(_df_arr):
            # Set measurement comb
            _fs_center = _fp_center / 2. - fNCO
            # Tune center frequency
            fs_center, df = lck.tune(_fs_center, df_val)
            # Listening comb. We listen at the signal, idler and half pump frequencies (and where we expect only noise)
            if nr_sig_freqs % 2 == 0:
                fs_comb = fs_center + df * np.arange(int(-nr_sig_freqs / 2), int(nr_sig_freqs / 2),1)
                print("Even number of frequencies (unbalanced for positive f.)",len(fs_comb),nr_sig_freqs)
            else:
                fs_comb = fs_center + df * np.arange(int(-nr_sig_freqs / 2), int(nr_sig_freqs / 2)+1,1)
                print("Odd number of frequencies",len(fs_comb),nr_sig_freqs)

            # Set pump frequency
            fp_center = 2 * (fs_center + fNCO) - fNCO_pump
            fp_comb = fp_center + df * k_array

            # Set df
            lck.set_df(df)

            # Save tuned df and the frequency combs
            df_arr[df_idx] = df
            fs_comb_arr[df_idx] = fs_comb
            fp_comb_arr[df_idx] = fp_comb
            fp_center_arr[df_idx] = fp_center

            # Set pump frequencies
            og_pump.set_frequencies(fp_comb)

            # Set signal frequencies
            ig.set_frequencies(fs_comb)

            # Set input phases
            input_phases[0]=0.0
            for i in range(1, nr_sig_freqs):
                input_phases[i] = input_phases[i-1] - dPhiDrift_df * df  # the minus work
            input_phases_q =  input_phases - np.pi /2 
            ig.set_phases(phases=input_phases.copy()) 

            # Pump amplitude loop
            for amp_idx, amp_val in enumerate(pump_amp_arr):
                # Pump amplitude
                og_pump.set_amplitudes(np.full(nr_pump_freqs, amp_val))

                # Pump 1 phase loop
                for phase_idx, phase_val in enumerate(pump_phase_arr):
                    phases_arr             = np.array([0])   #only one phase configuration
                    phases_q_arr           = phases_arr - np.pi/2
                    # phases_arr = np.zeros(nr_pump_freqs)
                    # phases_arr[0] = phase_val
                    # phases_q_arr = np.full(nr_pump_freqs, -np.pi / 2)
                    # phases_q_arr[0] = phase_val - np.pi / 2
                    og_pump.set_phases(phases=phases_arr,
                                       phases_q=phases_q_arr)

                    # Signal frequency loop
                    for sig_idx, sig_val in enumerate(fs_comb):
                        # Probe frequency
                        og.set_frequencies(sig_val)

                        lck.apply_settings()

                        # Get lock-in packets (pixels) from the local buffer
                        Npix = math.floor(Npix_arr[df_idx])
                        data = lck.get_pixels(Npix + Nskip, summed=False, nsum=Navg)
                        freqs, pixels_i, pixels_q = data[input_port]

                        # Convert a measured IQ pair into a low/high sideband pair
                        LSB, HSB = utils.untwist_downconversion(pixels_i, pixels_q)

                        usb_arr[df_idx, amp_idx, phase_idx, sig_idx] = np.mean(HSB[-Npix:], axis=0)

                        # Update progress bar
                        pbar.update(1)

    # Mute outputs at the end of the sweep
    og_pump.set_amplitudes(np.zeros(nr_pump_freqs))
    og.set_amplitudes(0.0)
    lck.apply_settings()
    lck.hardware.set_dc_bias(0.0, bias_port)


# Stop timer
t_end = time.strftime("%Y-%m-%d_%H_%M_%S")

# Create dictionary with attributes
myrun_attrs = {"Meas": meas_type,
               "Instr": Box,
               "T": temperature,
               "Sample": sample,
               "att": atten,
               "4K-amp_out": 42,
               "RT-amp_out": 41,
               "RT-amp_in": 0,
               "nr_sig_freqs": nr_sig_freqs,
               "nr_pump_freqs": nr_pump_freqs,
               "fp_center": fp_center_arr + fNCO_pump,
               "amp_sig": amp_sig,
               "DC bias": bias_val,
               "Npixels": Npix_arr,
               "Naverages": Navg,
               "Nskip": Nskip,
               "Dither": dither,
               "t_start": t_start,
               "t_meas": t_end,
               #"Script name": os.path.basename(__file__),
               }

# # Save attributes
save_attr(save_folder, save_file, meas_type, myrun, myrun_attrs)

# Save data
save_data(save_folder, save_file, meas_type, myrun, fs_comb_arr + fNCO, fp_comb_arr + fNCO_pump, pump_amp_arr,
          pump_phase_arr, df_arr, usb_arr)
print('Done.')

Presto version: 2.16.0


  0%|                                                  | 0/1140 [00:00<?, ?it/s]

Odd number of frequencies 95 95


 33%|█████████████▎                          | 380/1140 [02:26<04:53,  2.59it/s]

Odd number of frequencies 95 95


 67%|██████████████████████████▋             | 760/1140 [05:55<03:27,  1.83it/s]

Odd number of frequencies 95 95


100%|███████████████████████████████████████| 1140/1140 [10:26<00:00,  1.82it/s]

Saved run attributes.
Done.


In [4]:
# myrun       = time.strftime("%Y-%m-%d_%H_%M_%S")
# Save data
# save_data(save_folder, save_file, meas_type, myrun, fs_comb_arr + fNCO, fp_comb_arr + fNCO_pump, pump_amp_arr,
#           pump_phase_arr, df_arr, usb_arr)


## TESTING
print(df,fs_center+fNCO,_fs_center+fNCO)
print('fs_comb:',(fs_comb+fNCO)/df)
print('fNCO',fNCO)
print('fp_comb:',(fp_comb+fNCO_pump)/df)




7499.962500187499 4424854425.727871 4424855567.0
fs_comb: [589936.54 589937.54 589938.54 589939.54 589940.54 589941.54 589942.54
 589943.54 589944.54 589945.54 589946.54 589947.54 589948.54 589949.54
 589950.54 589951.54 589952.54 589953.54 589954.54 589955.54 589956.54
 589957.54 589958.54 589959.54 589960.54 589961.54 589962.54 589963.54
 589964.54 589965.54 589966.54 589967.54 589968.54 589969.54 589970.54
 589971.54 589972.54 589973.54 589974.54 589975.54 589976.54 589977.54
 589978.54 589979.54 589980.54 589981.54 589982.54 589983.54 589984.54
 589985.54 589986.54 589987.54 589988.54 589989.54 589990.54 589991.54
 589992.54 589993.54 589994.54 589995.54 589996.54 589997.54 589998.54
 589999.54 590000.54 590001.54 590002.54 590003.54 590004.54 590005.54
 590006.54 590007.54 590008.54 590009.54 590010.54 590011.54 590012.54
 590013.54 590014.54 590015.54 590016.54 590017.54 590018.54 590019.54
 590020.54 590021.54 590022.54 590023.54 590024.54 590025.54 590026.54
 590027.54 590028.5